In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import mne
import pickle
import scipy
from delphos_detector import *

In [ ]:
n_chs = 1
n_secs = 10
sfreq = 256
fs_range = (1, 100)
amplitude_range = (2, 5)
n_fs_bands = 10
n = 10000

In [ ]:
with open(f'n_{n}_chs_{n_chs}_fs_{sfreq}_t_{n_secs}.pickle', 'rb') as f:
  dataset = pickle.load(f)
X = np.array(dataset['X'])
Y = np.array(dataset['Y'])

Oct, NbVoi, VanMom = initialize_variables(sfreq)

NbVoi = 2

X_tf = []
X_tfz = []

for i in range(n):

    if not(i % 100):
        print(i)

    tf, freqs = DoG(
        np.squeeze(X[i]),
        Oct,
        NbVoi,
        VanMom,
        sfreq,
    )
    X_tf.append(tf.T)

    tfz, _, _, _ = z_H0(tf, sfreq) # apply zH0
    X_tfz.append(tfz)

del X

X_tf = np.array(X_tf)
X_tfz = np.array(X_tfz)

print(freqs)

In [ ]:
dataset = {'X': np.array(X_tf), 'Y': Y}
with open(f'n_{n}_chs_{n_chs}_fs_{sfreq}_t_{n_secs}_tf.pickle', 'wb') as f:
  pickle.dump(dataset, f)
del X_tf

In [ ]:
dataset = {'X': np.array(X_tfz), 'Y': Y}
with open(f'n_{n}_chs_{n_chs}_fs_{sfreq}_t_{n_secs}_tfz.pickle', 'wb') as f:
    pickle.dump(dataset, f)
del X_tfz, Y

In [ ]:
plt.rcParams["figure.figsize"] = (20,3)

channel = 1
Oct, NbVoi, VanMom = initialize_variables(sfreq)
NbVoi = 2
i = 1
tf, freqs = DoG(np.squeeze(X[i]), Oct, NbVoi,VanMom,sfreq,)
tfz, _, _, _ = z_H0(tf, sfreq)

print(freqs)

timepoint = np.random.randint(200, 2560-200)
print(timepoint)
samples = range(timepoint-200, timepoint+200)
plt.imshow(np.real(tf[:, samples]), cmap=plt.get_cmap("turbo"), vmin= np.min(np.real(tf)), vmax=np.max(np.real(tf)))
plt.plot(X[channel, samples])
plt.colorbar()
plt.show()

plt.imshow(tfz.T[:, samples], cmap=plt.get_cmap("turbo"), vmin= np.min(np.real(tfz)), vmax=np.max(np.real(tfz)))
plt.plot(X[channel, samples])